In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical

In [6]:
test_data = pd.read_csv("../05machine_learning/data/bank_app_reviews_test.csv")
test_data.head(2)

,리뷰일,평점,사용자리뷰,업체답변,은행명
0,2024-02-08,5,고경민계장님감사해요,"안녕하세요 최순녀 고객님. 칭찬 진심으로 감사드리며, 더욱 편리하고 안정적인 서비스...",우리
1,2023-07-24,5,저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...,"신아​ 님, 안녕하세요? 뱅크샐러드 고객감동팀​입니다. 소중한 시간내어 고객센터에 ...",뱅크샐러드


In [7]:
import re

def clean_text(text):
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9\s]','', text) #한글, 영문, 숫자
    cleaned = re.sub(r'\s+', ' ', cleaned) # 연속된 공백을 하나의 공백
    return cleaned.strip()

In [8]:
test_data['사용자리뷰'] = test_data['사용자리뷰'].apply(clean_text)
test_data['사용자리뷰']

0                                              고경민계장님감사해요
1       저축목표피드 새로 생긴거 너무좋은데 분명 카테고리를 저축으로 했는데 왜 인식이 안되...
2         아니 이딴걸 편리하게 사용하는앱이라고 쳐만들엇나 이렇게 불편하게만든건 일부러그런거에요
3       몇 년째 만족하며 사용중이라 조금식 개선되어거는 모습에 만족하며 사용중입니다 하지만...
4                                스타뱅킹을 사용 하고나서부터 편안해서 좋아요
                              ...                        
9529    만보기 이벤트는 실망스러워요 후기 말투 다 똑같고 사기 맞죠 양심이 참 정직하게 확...
9530                   기능이 많아 다 사용해보진 못 했지만 대체적으로 편한거 같아요
9531                                                편리하네요
9532                                            사용하기 편리해요
9533    너무 후져서 오랜만에 어플이용하다가 욕했답니다 인증방식이 2010년대에 머물러계심 ...
Name: 사용자리뷰, Length: 9534, dtype: object

In [9]:
test_data['is_good'] = test_data['평점'].apply(lambda x: 1 if x >=4 else 0)
test_data['is_good']

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

In [10]:
mecab = Mecab()

In [11]:
tokenized_docs = test_data['사용자리뷰'].apply(mecab.morphs)

In [12]:
tokenized_docs[0]

['고경민', '계장', '님', '감사', '해요']

# train에서 사용했던 tokenizer를 불러와서 one hot encoding

In [17]:
import joblib

In [18]:
token = joblib.load("./model/bank_app_tokeizer.joblib")

In [19]:
x = token.texts_to_sequences(tokenized_docs)
print(x[0])

[6248, 327, 111, 71]


# train에서 사용했던 패딩 길이(모델에 넣을 컬럼 수)

In [20]:
max_length = joblib.load("./model/bank_app_max_length.joblib")
print(max_length)

302


In [21]:
X_padded = pad_sequences(x, maxlen=max_length, padding='post')
print(X_padded[1])

[ 216  717  370 1524   39   44    8  148 1025  724   27 1033   31   43
   14   51  117    3    6    9    4  861  117    9  414  224  112    9
  164  409 2261 1336   20    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [22]:
len(X_padded[1])

302

In [23]:
y = test_data['is_good']
y

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

# 모델 불러와서 예측하고 결과 비교하기

In [24]:
birnn_best = load_model("./model/bank_app_review_birnn.keras")
cnn_lstm_best = load_model("./model/bank_app_review_lstm_cnn.keras")
attn_best = load_model("./model/bank_app_review_attn_model.keras")

I0000 00:00:1747887848.961304    9142 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3065 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [25]:
birnn_pred = birnn_best.predict(X_padded)
cnn_latm_pred = cnn_lstm_best.predict(X_padded)
attn_pred = attn_best.predict(X_padded)

I0000 00:00:1747887951.154436    9400 service.cc:152] XLA service 0x7f9ae8004c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747887951.154582    9400 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-05-22 13:25:51.206149: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747887951.278324    9400 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/298 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step   

I0000 00:00:1747887951.551901    9400 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


298/298 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step
298/298 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step
298/298 ━━━━━━━━━━━━━━━━━━━━ 30s 93ms/step


In [28]:
birnn_pred = pd.DataFrame(birnn_pred)
cnn_lstm_pred = pd.DataFrame(cnn_latm_pred)
attn_pred = pd.DataFrame(attn_pred)

In [30]:
y

0       1
1       1
2       0
3       0
4       1
       ..
9529    0
9530    1
9531    1
9532    1
9533    0
Name: is_good, Length: 9534, dtype: int64

In [36]:
y = pd.DataFrame(y)

In [38]:
birnn_result = y.join(birnn_pred)
cnn_lstm_result = y.join(cnn_lstm_pred)
attn_pred_result = y.join(attn_pred)

In [42]:
birnn_result.loc[:, 0] = birnn_result.loc[:, 0].apply(lambda x: 1 if x > 0.5 else 0)
cnn_lstm_result.loc[:, 0] = cnn_lstm_result.loc[:, 0].apply(lambda x: 1 if x > 0.5 else 0)
attn_pred_result.loc[:, 0] = attn_pred_result.loc[:, 0].apply(lambda x: 1 if x > 0.5 else 0)


In [45]:
birnn_result

,is_good,0
0,1,1.0
1,1,0.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [43]:
cnn_lstm_result

,is_good,0
0,1,1.0
1,1,0.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [44]:
attn_pred_result

,is_good,0
0,1,1.0
1,1,0.0
2,0,0.0
3,0,1.0
4,1,1.0
...,...,...
9529,0,0.0
9530,1,1.0
9531,1,1.0
9532,1,1.0


In [46]:
from sklearn.metrics import classification_report

In [47]:
print(classification_report(birnn_result['is_good'], birnn_result[0]))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87      3862
           1       0.94      0.87      0.90      5672

    accuracy                           0.89      9534
   macro avg       0.88      0.89      0.89      9534
weighted avg       0.89      0.89      0.89      9534



In [48]:
print(classification_report(cnn_lstm_result['is_good'], cnn_lstm_result[0]))

              precision    recall  f1-score   support

           0       0.84      0.91      0.88      3862
           1       0.94      0.88      0.91      5672

    accuracy                           0.89      9534
   macro avg       0.89      0.90      0.89      9534
weighted avg       0.90      0.89      0.90      9534



In [49]:
print(classification_report(attn_pred_result['is_good'], attn_pred_result[0]))

              precision    recall  f1-score   support

           0       0.86      0.90      0.88      3862
           1       0.93      0.90      0.91      5672

    accuracy                           0.90      9534
   macro avg       0.89      0.90      0.90      9534
weighted avg       0.90      0.90      0.90      9534



In [ ]:
attn_pred_result

# evaluate

In [51]:
%%time
birnn_best.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.8900 - auc: 0.9494 - loss: 0.2806
CPU times: user 3.37 s, sys: 808 ms, total: 4.18 s
Wall time: 7.22 s


[0.2848619818687439, 0.8879798650741577, 0.9476749300956726]

In [52]:
%%time
cnn_lstm_best.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.8980 - auc: 0.9565 - loss: 0.2565
CPU times: user 3.77 s, sys: 1.17 s, total: 4.94 s
Wall time: 4.58 s


[0.26050737500190735, 0.8949024677276611, 0.9555007219314575]

In [53]:
%%time
attn_best.evaluate(X_padded, test_data['is_good'])

298/298 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.9013 - auc: 0.9585 - loss: 0.2600
CPU times: user 4.96 s, sys: 2.22 s, total: 7.17 s
Wall time: 9.22 s


[0.26245906949043274, 0.899937093257904, 0.9579288363456726]